In [1]:
import pandas as pd

In [4]:
df = pd.read_csv("LinkedinCompanies - csv - LinkedinCompanies.csv", sep=',')
display(df)

,Nome da Empresa,Categoria,website,Qtd Funcionários,Data da Fundação,Cidade,UF
0,RDO App | Diário de Obras como Ferramenta Lean...,Engenharia civil,http://www.rdoapp.com.br,2-10,2018,Monte Mor,SP
1,Pashal Locadora de Equipamentos,Construção,http://www.pashal.com,201-500,1990,São José do Rio Preto,SP
2,Universidade Estadual Vale do Acaraú - UVA,"Sobral, Ceará",http://www.uvanet.br/,201-500,1968,Sobral,CE
3,MBigucci Construtora,Imobiliário,http://www.mbigucci.com.br,51-200,1983,São Bernardo do Campo,SP
4,Sobrosa Construtora,Construção,http://www.sobrosa.com.br,201-500,1989,Travessa Ubirassanga 46,SP
...,...,...,...,...,...,...,...
2168,Bley Performance Digital,Imobiliário,https://bleyperformancedigital.com.br,2-10,2020,Rio de Janeiro,RJ
2169,ROS IMOVEIS - Desde 1976,Imobiliário,http://www.rosimoveis.com.br,11-50,1976,Blumenau,SC
2170,Lucas Santiago,Imobiliário,https://www.bhmgimoveis.com.br/,2-10,Sem informação,Belo Horizonte,MG
2171,DM8 Assessoria & Negócios Imobiliários,Imobiliário,http://www.dm8imobiliaria.com.br,2-10,2010,Campinas,SP


# Fazendo a Limpeza e tratamento dos dados

In [5]:
companies = df.copy()

Há 16 linhas onde não foi possível obter o endereço da empresa ou a mesma não preencheu


In [6]:
companies.isnull().sum()

Nome da Empresa      0
Categoria            0
website              0
Qtd Funcionários     0
Data da Fundação     0
Cidade               0
UF                  67
dtype: int64

Empresas que não tiveram suas localizações encontradas

In [7]:
companies[companies['UF'].isnull()]

,Nome da Empresa,Categoria,website,Qtd Funcionários,Data da Fundação,Cidade,UF
11,CONSLOG - Software de Gestão de Obras,Software,http://www.conslog.com,11-50,2012,Sem informação,NaN
43,PGMAK Projetos e Gerenciamento Ltda,Construção,http://www.pgmak.com.br,11-50,2009,Sem informação,NaN
49,GUINDASTEC - Guindastes e Serviços Técnicos LTDA,Maquinário,http://www.guindastec.com.br,51-200,1979,Sem informação,NaN
53,JT Engenharia & Projeto,Engenharia mecânica ou industrial,http://www.jtengenharia.eng.br,2-10,2016,Sem informação,NaN
93,"TDI Global Solutions, Inc.",Embalagens e recipientes,http://www.tdigs.com,51-200,2006,Sem informação,NaN
...,...,...,...,...,...,...,...
1906,Flavio Valle Consultoria Imobiliária Premium,Imobiliário,http://www.flaviovalle.com,2-10,2018,Sem informação,NaN
1920,TERRA IMÓVEIS RJ,Imobiliário,http://www.terraimoveisrj.com.br/,11-50,2018,Sem informação,NaN
1967,Youmob.com.br,Imobiliário,http://youmob.com.br,0,Sem informação,Sem informação,NaN
1995,LG Consultoria Imobiliária e Turismo,Imobiliário,http://www.lidianegomes.com,2-10,2012,Sem informação,NaN


 Após analisar empresa a empresa peguei as informações manualmente e inseri em uma nova planilha

In [8]:
df2 = pd.read_csv("LinkedinCompanies - teste.csv")
display(df2)

,Nome da Empresa,Categoria,website,Qtd Funcionários,Data da Fundação,Localização
0,RDO App | Diário de Obras como Ferramenta Lean...,Engenharia civil,http://www.rdoapp.com.br,2-10,2018,"São Paulo, Monte Mor, São Paulo 13190-000,"
1,Pashal Locadora de Equipamentos,Construção,http://www.pashal.com,201-500,1990,"Rua Dionizio Zacaron, São José do Rio Preto, S..."
2,Universidade Estadual Vale do Acaraú - UVA,"Sobral, Ceará",http://www.uvanet.br/,201-500,1968,"Av. da Universidade, 850 - Campus da Betânia C..."
3,MBigucci Construtora,Imobiliário,http://www.mbigucci.com.br,51-200,1983,"Av. Senador Vergueiro, 3597 - 9º andar, Rudge ..."
4,Sobrosa Construtora,Construção,http://www.sobrosa.com.br,201-500,1989,"Travessa Ubirassanga 46, São Paulo, 04614-050,"
...,...,...,...,...,...,...
2171,Bley Performance Digital,Imobiliário,https://bleyperformancedigital.com.br,2-10,2020,"Rua Marquês de Olinda 61, Rio de Janeiro, RJ 2..."
2172,ROS IMOVEIS - Desde 1976,Imobiliário,http://www.rosimoveis.com.br,11-50,1976,"Rua Namy Deeke 99, Loja Terrea , Blumenau, San..."
2173,Lucas Santiago,Imobiliário,https://www.bhmgimoveis.com.br/,2-10,Sem informação,"Belo Horizonte,"
2174,DM8 Assessoria & Negócios Imobiliários,Imobiliário,http://www.dm8imobiliaria.com.br,2-10,2010,"Rua Professor Manoel Saturnino do Amaral, Conj..."


## Fazendo o request dos municípios e Estados do Brasil

In [9]:
import requests as r
import io

In [10]:
municipios = r.get("http://blog.mds.gov.br/redesuas/wp-content/uploads/2018/06/Lista_Munic%C3%ADpios_com_IBGE_Brasil_Versao_CSV.csv")

estados = r.get("http://blog.mds.gov.br/redesuas/wp-content/uploads/2018/06/Lista_Estados_Brasil_Versao_CSV.csv")

In [11]:
df_municipios = pd.read_csv(io.StringIO(municipios.text), sep=";")
df_estados = pd.read_csv(io.StringIO(estados.text), sep = ";")

display(df_municipios)
display(df_estados)

,ConcatUF+Mun,IBGE,IBGE7,UF,Município,Região,População 2010,Porte,Capital,Unnamed: 9
0,ROAlta Floresta D'oeste,110001,1100015,RO,Alta Floresta D´oeste,Região Norte,24392.0,Pequeno II,NaN,PROCV(A2;'[Lista de Municípios com IBGE - Bras...
1,ROAriquemes,110002,1100023,RO,Ariquemes,Região Norte,90353.0,Médio,NaN,PROCV(A2;'Lista de Municípios IBGE'!$A$2:$G$55...
2,ROCabixi,110003,1100031,RO,Cabixi,Região Norte,6313.0,Pequeno I,NaN,NaN
3,ROCacoal,110004,1100049,RO,Cacoal,Região Norte,78574.0,Médio,NaN,NaN
4,ROCerejeiras,110005,1100056,RO,Cerejeiras,Região Norte,17029.0,Pequeno I,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
5565,GOVianópolis,522200,5222005,GO,Vianópolis,Região Centro-Oeste,12548.0,Pequeno I,NaN,NaN
5566,GOVicentinópolis,522205,5222054,GO,Vicentinópolis,Região Centro-Oeste,7371.0,Pequeno I,NaN,NaN
5567,GOVila Boa,522220,5222203,GO,Vila Boa,Região Centro-Oeste,4735.0,Pequeno I,NaN,NaN
5568,GOVila Propício,522230,5222302,GO,Vila Propício,Região Centro-Oeste,5145.0,Pequeno I,NaN,NaN


,IBGE,Estado,UF,Região,Qtd Mun,Sintaxe,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,11,Rondônia,RO,Região Norte,52,11'RO',NaN,NaN,PROCV(A2;'Lista de Estados IBGE'!$A$2:$C$28;2;0)
1,12,Acre,AC,Região Norte,22,12'AC',NaN,NaN,NaN
2,13,Amazonas,AM,Região Norte,62,13'AM',NaN,NaN,NaN
3,14,Roraima,RR,Região Norte,15,14'RR',NaN,NaN,NaN
4,15,Pará,PA,Região Norte,144,15'PA',NaN,NaN,NaN
5,16,Amapá,AP,Região Norte,16,16'AP',NaN,NaN,NaN
6,17,Tocantins,TO,Região Norte,139,17'TO',NaN,NaN,NaN
7,21,Maranhão,MA,Região Nordeste,217,21'MA',NaN,NaN,NaN
8,22,Piauí,PI,Região Nordeste,224,22'PI',NaN,NaN,NaN
9,23,Ceará,CE,Região Nordeste,184,23'CE',NaN,NaN,NaN


In [12]:
list_municipios = df_municipios['Município'].to_list()
list_estados = df_estados['Estado'].to_list()

In [13]:
list_estados

['Rondônia',
 'Acre',
 'Amazonas',
 'Roraima',
 'Pará',
 'Amapá',
 'Tocantins',
 'Maranhão',
 'Piauí',
 'Ceará',
 'Rio Grande do Norte',
 'Paraíba',
 'Pernambuco',
 'Alagoas',
 'Sergipe',
 'Bahia',
 'Minas Gerais',
 'Espírito Santo',
 'Rio de Janeiro',
 'São Paulo',
 'Paraná',
 'Santa Catarina',
 'Rio Grande do Sul',
 'Mato Grosso do Sul',
 'Mato Grosso',
 'Goiás',
 'Distrito Federal']

In [14]:
list_municipios

['Alta Floresta D´oeste',
 'Ariquemes',
 'Cabixi',
 'Cacoal',
 'Cerejeiras',
 'Colorado do Oeste',
 'Corumbiara',
 'Costa Marques',
 'Espigão D´oeste',
 'Guajará-mirim',
 'Jaru',
 'Ji-paraná',
 'Machadinho D´oeste',
 'Nova Brasilândia D´oeste',
 'Ouro Preto do Oeste',
 'Pimenta Bueno',
 'Porto Velho',
 'Presidente Médici',
 'Rio Crespo',
 'Rolim de Moura',
 'Santa Luzia D´oeste',
 'Vilhena',
 'São Miguel do Guaporé',
 'Nova Mamoré',
 'Alvorada D´oeste',
 'Alto Alegre Dos Parecis',
 'Alto Paraíso',
 'Buritis',
 'Novo Horizonte do Oeste',
 'Cacaulândia',
 'Campo Novo de Rondônia',
 'Candeias do Jamari',
 'Castanheiras',
 'Chupinguaia',
 'Cujubim',
 'Governador Jorge Teixeira',
 'Itapuã do Oeste',
 'Ministro Andreazza',
 'Mirante da Serra',
 'Monte Negro',
 'Nova União',
 'Parecis',
 'Pimenteiras do Oeste',
 'Primavera de Rondônia',
 'São Felipe D´oeste',
 'São Francisco do Guaporé',
 'Seringueiras',
 'Teixeirópolis',
 'Theobroma',
 'Urupá',
 'Vale do Anari',
 'Vale do Paraíso',
 'Acrelân

In [15]:
companies = df2.copy()

In [82]:
companies_local = companies['Localização'].to_list()
words = [word.split()for word in companies_local]

In [103]:
words_join = [" ".join(word) for word in words]
words_join

['São Paulo, Monte Mor, São Paulo 13190-000,',
 'Rua Dionizio Zacaron, São José do Rio Preto, São Paulo 15077-010,',
 'Av. da Universidade, 850 - Campus da Betânia CEP. 62.040-370 - Sobral - Ceará, Sobral, Ceará',
 'Av. Senador Vergueiro, 3597 - 9º andar, Rudge Ramos, São Bernardo do Campo, SP',
 'Travessa Ubirassanga 46, São Paulo, 04614-050,',
 'Rua Serra de Bragança, 1.814, São Paulo, SP 03318-000,',
 'Rua Des. Leite Albuquerque, 635, Fortaleza, Ceará 60150-150,',
 'Rua Creso Barbi 96, Belo Horizonte, Minas Gerais 31340-070,',
 'Alameda Jauaperi 897, São Paulo,',
 'Rua Manuel da Nóbrega, nº 354 12ª Bairro Paraíso, São Paulo, SP 04001001,',
 'R, Bauru, SP 17015300,',
 'Sem informação',
 'Avenida Doutor Carlos Botelho, 2724, Sala 5, São Carlos, São Paulo 13560-251,',
 'Rua Sergipe, 1440, Savassi, 11o Andar, Belo Horizonte, Minas Gerais 30.130-174,',
 'São Paulo, SP 01533010,',
 'Rua Cirne Maia , sala 604, Rio de janeiro , Rio de janeiro 22771410,',
 'Alameda Flávia 215, Valinhos, São 

In [142]:
for word in words_join:
    if "são paulo " in word.lower():
        print("SP",word)
    

SP São Paulo, Monte Mor, São Paulo 13190-000,
SP Rua Dionizio Zacaron, São José do Rio Preto, São Paulo 15077-010,
SP Avenida Doutor Carlos Botelho, 2724, Sala 5, São Carlos, São Paulo 13560-251,
SP Alameda Flávia 215, Valinhos, São Paulo 13278-180,
SP Rua Alvorada 1289, São Paulo, São Paulo 04550004,
SP Rua Visconde do Rio Branco, 726, Centro, São João da Boa Vista, São Paulo 13870-010,
SP Rua Alexandre Dumas 545, Sala 03, São Paulo, São Paulo 04717-000,
SP São José dos Campos, São Paulo 12245-001,
SP Avenida Maurilio Biagi 800, Ribeirão Preto, São Paulo 14021-140,
SP Avenida Paulista 91, Sala 905, São Paulo, São Paulo 01311-000,
SP Alameda Xingu 350, 12° andar, Barueri, São Paulo 06455-030,
SP Avenida 02, 1807, Jardim Claret, Rio Claro, São Paulo 13500411,
SP Rua Borges Lagoa, 971, São Paulo, São Paulo 04038-032,
SP Av. João Paulo I, 770, Embu das Artes, São Paulo 06817-000,
SP Av. Brigadeiro Faria Lima, 1.461 – Torre Sul 4º andar – Cj. 41 - Pinheiros, São Paulo, São Paulo 01452-002,

In [121]:
stopwords = ["Av.", "CEP", "1","2","3", "4", "5", "6", "7", "8", "9", "0", "-", "," "." , ",,", "Rua"]
word_treat=list()
for word in words_join:
    stopwords.append(("".join(filter(lambda x: x.lower() in stopwords ,  word))))

stopwords

    

['Av.',
 'CEP',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '0',
 '-',
 ',.',
 ',,',
 'Rua',
 '13190-000',
 '15077-010',
 '850-62040-370--',
 '3597-9',
 '4604614-050',
 '181403318-000',
 '63560150-150',
 '9631340-070',
 '897',
 '3541204001001',
 '17015300',
 '',
 '2724513560-251',
 '14401130130-174',
 '01533010',
 '60422771410',
 '21513278-180',
 '43558038-230',
 '118190330140-091',
 '96888015-600',
 '542-1001310-000',
 '128904550004',
 '96888015-600',
 '336',
 '5488053645',
 '13500330',
 '05042-010',
 '33824040280',
 '13206-765',
 '',
 '156549025-330',
 '133363334380020-270',
 '1431386047-480',
 '72613870-010',
 '5450304717-000',
 '-45318995001-970',
 '20038402-322',
 '',
 '',
 '296-3-87014-100',
 '12245-001',
 '37904063-000',
 '01031-001',
 '',
 '80014021-140',
 '49203066-065',
 '9190501311-000',
 '3501206455-030',
 '2380',
 '',
 '377-2701009-000',
 '',
 '32670408',
 '',
 '148100420031142',
 '02180713500411',
 '',
 '97104038-032',
 '7302856-050',
 '167294940030',
 '540-050

In [122]:
word_treat=list()
stopwords.append("Avenida")
for word in words_join:
    word_treat.append(("".join(filter(lambda x: x.lower() not in stopwords ,  word))))

word_treat

['São Paulo, Monte Mor, São Paulo ,',
 'Rua Dionizio Zacaron, São José do Rio Preto, São Paulo ,',
 'Av. da Universidade,   Campus da Betânia CEP. .  Sobral  Ceará, Sobral, Ceará',
 'Av. Senador Vergueiro,   º andar, Rudge Ramos, São Bernardo do Campo, SP',
 'Travessa Ubirassanga , São Paulo, ,',
 'Rua Serra de Bragança, ., São Paulo, SP ,',
 'Rua Des. Leite Albuquerque, , Fortaleza, Ceará ,',
 'Rua Creso Barbi , Belo Horizonte, Minas Gerais ,',
 'Alameda Jauaperi , São Paulo,',
 'Rua Manuel da Nóbrega, nº  ª Bairro Paraíso, São Paulo, SP ,',
 'R, Bauru, SP ,',
 'Sem informação',
 'Avenida Doutor Carlos Botelho, , Sala , São Carlos, São Paulo ,',
 'Rua Sergipe, , Savassi, o Andar, Belo Horizonte, Minas Gerais .,',
 'São Paulo, SP ,',
 'Rua Cirne Maia , sala , Rio de janeiro , Rio de janeiro ,',
 'Alameda Flávia , Valinhos, São Paulo ,',
 'Av. MAnoel Morais, , Manaíra, João Pessoa, Paraíba ,',
 'Rua Gonçalves Dias, , Sala , Belo Horizonte, Minas Gerais ,',
 'Rua Almirante Lamego , Flori

In [10]:
lista = [[1,2,3],[4,5,6]]
list_values = []
for value in lista:
    for value_list in value:
        list_values.append(value_list)
list_values
data_frame = pd.DataFrame(data= {"teste": list_values})
display(data_frame)

,teste
0,1
1,2
2,3
3,4
4,5
5,6
